In [1]:
import tqdm
import torch
import wandb
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW

C:\Users\Sid\AppData\Local\Temp\ipykernel_15356\451788269.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
wandb.init(project="protienSequence Rah!!!")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: siddiqmoideen07. Use `wandb login --relogin` to force relogin


In [4]:
data = pd.read_csv("./proteinSeq/win_7.csv")
data

,Windowed_Sequence,Solvent_Accessibility,Solvent_Accessibility_Length,Primary_Sequence_0,Primary_Sequence_1,Primary_Sequence_2,Primary_Sequence_3,Primary_Sequence_4,Primary_Sequence_5,Primary_Sequence_6,Solvent_Accessibility_0,Solvent_Accessibility_1,Solvent_Accessibility_2,Solvent_Accessibility_3,Solvent_Accessibility_4,Solvent_Accessibility_5,Solvent_Accessibility_6
0,HRFKVYN,BEFEEEE,7,6,14,4,8,17,19,11,1,0,2,0,0,0,0
1,RFKVYNY,EFEEEEF,7,14,4,8,17,19,11,19,0,2,0,0,0,0,2
2,FKVYNYM,FEEEEFE,7,4,8,17,19,11,19,10,2,0,0,0,0,2,0
3,KVYNYMS,EEEEFEE,7,8,17,19,11,19,10,15,0,0,0,0,2,0,0
4,VYNYMSP,EEEFEEE,7,17,19,11,19,10,15,12,0,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236679,FESSSGL,FFEEFEF,7,4,3,15,15,15,5,9,2,2,0,0,2,0,2
236680,ESSSGLV,FEEFEFE,7,3,15,15,15,5,9,17,2,0,0,2,0,2,0
236681,SSSGLVW,EEFEFEB,7,15,15,15,5,9,17,18,0,0,2,0,2,0,1
236682,SSGLVWT,EFEFEBE,7,15,15,5,9,17,18,16,0,2,0,2,0,1,0


In [10]:
cls=list(set(data['Solvent_Accessibility_0']))

In [11]:
cls

[0, 1, 2]

In [15]:
labEncoder=LabelEncoder()
labEncoder.fit(cls)
clsDic=dict(zip(cls,labEncoder.transform(cls)))

In [16]:
clsDic

{0: 0, 1: 1, 2: 2}

In [17]:
numLabel=len(cls)
batchSize=16
learningRate=2e-5
epochs=200

In [18]:
device=torch.device("cuda") if torch.cuda.is_available else torch.device("cpu")
device

device(type='cuda')

In [19]:
model=AutoModelForSequenceClassification.from_pretrained("google/mobilebert-uncased",num_labels=numLabel).to(device)
token=AutoTokenizer.from_pretrained("google/mobilebert-uncased")

c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, u

In [20]:
wandb.watch(model)
wandb.config["learningRate"]=learningRate
wandb.config["batch_size"]=batchSize

In [ ]:
trainData,valData=train_test_split(data)